In [1]:
from pyspark.sql import SparkSession

In [2]:
# Set Up Spark Session
spark = SparkSession.builder.appName("Joy Ride").master("local[6]").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/07 23:13:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read data
path = "./data/202407-citibike-tripdata_5.csv"
df = spark.read.csv(path, header=True)
df

DataFrame[Unnamed: 0: string, ride_id: string, rideable_type: string, started_at: string, ended_at: string, start_station_name: string, start_station_id: string, end_station_name: string, end_station_id: string, start_lat: string, start_lng: string, end_lat: string, end_lng: string, member_casual: string]

In [5]:
df.take(100)

[Row(Unnamed: 0='4000000', ride_id='557426B1CE207702', rideable_type='electric_bike', started_at='2024-07-17 19:30:15.471', ended_at='2024-07-17 19:34:22.646', start_station_name='Sullivan St & Washington Sq', start_station_id='5721.01', end_station_name='E 12 St & 3 Ave', end_station_id='5788.12', start_lat='40.73047747', start_lng='-73.99906065', end_lat='40.73223272', end_lng='-73.98889957', member_casual='member'),
 Row(Unnamed: 0='4000001', ride_id='A60E366FEE8B51E8', rideable_type='electric_bike', started_at='2024-07-15 19:17:52.523', ended_at='2024-07-15 19:25:24.399', start_station_name='Sullivan St & Washington Sq', start_station_id='5721.01', end_station_name='Forsyth St & Grand St', end_station_id='5382.07', start_lat='40.73047747', start_lng='-73.99906065', end_lat='40.71779817737835', end_lng='-73.99316132068634', member_casual='member'),
 Row(Unnamed: 0='4000002', ride_id='ACA30207F80B5DF9', rideable_type='classic_bike', started_at='2024-07-27 13:34:48.531', ended_at='202